In [66]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import time

# Helper Clases / Functions

In [2]:
def Load_Data(num):
    if (num == -1): # All data
        X_all = []
        y_all = []
        for i in range(8):
            file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
            data = h5py.File(file_path, 'r')
            X = np.copy(data['image'])
            y = np.copy(data['type'])
            X = X[:, 0:23, :]
            X_all.append(X)
            y = y[0,0:X.shape[0]:1]
            y_all.append(y)
        A, N, E, T = np.shape(X_all)
        X_all = np.reshape(X_all, (A*N, E, T))
        y_all = np.reshape(y_all, (-1))
        y_all = y_all - 769
        ## Remove NAN
        index_Nan = []
        for i in range(A*N):
            for j in range(E):
                if (any(np.isnan(X_all[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X_all = np.delete(X_all, index_Nan, axis=0)
        y_all = np.delete(y_all, index_Nan)
        return (X_all, y_all)
    else:
        file_path = './../project_datasets/A0' + str(num) + 'T_slice.mat'
        data = h5py.File(file_path, 'r')
        X = np.copy(data['image'])
        y = np.copy(data['type'])
        X = X[:, 0:23, :]
        y = y[0,0:X.shape[0]:1]
        y = y - 769
         ## Remove NAN
        N, E, T = np.shape(X)
        index_Nan = []
        for i in range(N):
            for j in range(E):
                if (any(np.isnan(X[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X = np.delete(X, index_Nan, axis=0)
        y = np.delete(y, index_Nan)
        return (X, y)

# Load Data

In [71]:
start_time = time.time()
X, y = Load_Data(-1) # -1 to load all datas
print (np.shape(X))
print (y.shape)
print('runtime: ', time.time() - start_time)

(2280, 23, 1000)
(2280,)
runtime:  6.97817850112915


# Normalize Input

In [72]:
start_time = time.time()
#test = np.random.randint(10, size=(2,3,4))
#print(test)
#print(test.mean(axis=2, keepdims=True).shape)
#print(test.std(axis=2, keepdims=True).shape)
#normal = (test - test.mean(axis=2, keepdims=True))/(test.std(axis=2, keepdims=True))
#print(normal)

# normalize input to zero mean and unit variance
X = (X - X.mean(axis=2, keepdims=True))/(X.std(axis=2, keepdims=True))
print(X.shape)
print('runtime: ', time.time() - start_time)

(2280, 23, 1000)
runtime:  1.1409704685211182


# Add Derivative as Feature

In [73]:
# !!!This X will keep growing if you dont reload it first
start_time = time.time()
# take derivative
X_diff = np.diff(X, axis=2)

# zero pad to make same length as input
X_diff = np.concatenate((X_diff, np.zeros((X_diff.shape[0], X_diff.shape[1], 1))), axis=2)

# concatenate to with X to have more features
X = np.concatenate((X ,X_diff), axis=1)

# expect shape of (N, 2E, T)
print (X.shape)
N, E, T = np.shape(X)
print('runtime: ', time.time() - start_time)

(2280, 46, 1000)
runtime:  1.425959587097168


# Make DataLoaders

In [74]:
bs_train = 200
bs_val = 100
bs_test = 100
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dataloaders = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-bs_val-bs_test, bs_val, bs_test])
dataloaders['train'] = data_utils.DataLoader(dset['train'], batch_size=bs_train, shuffle=True, num_workers=1)
dataloaders['val'] = data_utils.DataLoader(dset['val'], batch_size=bs_val, shuffle=True, num_workers=1)
dataloaders['test'] = data_utils.DataLoader(dset['test'], batch_size=bs_test, shuffle=True, num_workers=1)

# Define Model

In [75]:
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer)
                            # batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        #h0 = Variable(torch.zeros(self.num_layer, x.size(1), self.hidden_dim)).cuda()
        #c0 = Variable(torch.zeros(self.num_layer, x.size(1), self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[-1, :, :]
        out = self.classifier(out)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            #out = self.forward(X_sample.cuda())
            out = self.forward(X_sample)
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

# Test the Model

In [ ]:
dtype = torch.FloatTensor
hidden_dim = 200
num_classes = 4
num_epoches = 10
model = myLSTM(E, hidden_dim, 1, num_classes)
#print(model.lstm.weight_ih_l0)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train)
        # print (out)
        loss = loss_fn(out, y_train.long())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
        

(0 batch) loss: 1.393788
(1 batch) loss: 1.387755
(2 batch) loss: 1.385640
(3 batch) loss: 1.393310
(4 batch) loss: 1.384565
